In [7]:
import numpy as np
import open3d as o3d
import math
import back_surface as back

In [8]:
def createPointCloud(points_list):
    temp = o3d.geometry.PointCloud()
    temp.points = o3d.utility.Vector3dVector(np.asarray(points_list))
    return temp

def computeNormal(points,cam):
    ##search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30)
    points.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.5, max_nn=9))
    points.orient_normals_towards_camera_location(camera_location=cam)

def mergeTwoClouds(fcloud,scloud):
    points = np.append(np.asarray(fcloud.points),np.asarray(scloud.points),axis=0)
    normals = np.append(np.asarray(fcloud.normals),np.asarray(scloud.normals),axis=0)
    result = createPointCloud(points)
    result.normals = o3d.utility.Vector3dVector(normals)
    return result

## Main

In [9]:
for i in range(1,9):
    pcd = o3d.io.read_point_cloud("./Points/clean points/dwnsample_pose_"+str(i)+".ply")

    # inverse the surface
    points = np.asarray(pcd.points)
    behi = back.generateBack(points,0.05,0.1,0.05)
    bcd = createPointCloud(behi) #back point cloud

    # compute normal vectors
    cam_front = np.array([0,0,0])
    cam_back = np.array([0,0,10])
    computeNormal(pcd,cam_front)
    computeNormal(bcd,cam_back)
    #o3d.visualization.draw_geometries([pcd,bcd])

    #merge 2 surfaces
    merge = mergeTwoClouds(pcd,bcd)
    #o3d.visualization.draw_geometries([merge])

    print('run Poisson surface reconstruction')
    mesh, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(merge, depth=8)
    print(mesh)
    mesh.compute_vertex_normals()
    #o3d.visualization.draw_geometries([mesh])

    o3d.io.write_triangle_mesh('./Mesh/Poisson Mesh/normal/pose_'+str(i)+'.ply',mesh)

run Poisson surface reconstruction
geometry::TriangleMesh with 3539 points and 7056 triangles.
run Poisson surface reconstruction
geometry::TriangleMesh with 3483 points and 6938 triangles.
run Poisson surface reconstruction
geometry::TriangleMesh with 3410 points and 6805 triangles.
run Poisson surface reconstruction
geometry::TriangleMesh with 3501 points and 6984 triangles.
run Poisson surface reconstruction
geometry::TriangleMesh with 3331 points and 6630 triangles.
run Poisson surface reconstruction
geometry::TriangleMesh with 3492 points and 6941 triangles.
run Poisson surface reconstruction
geometry::TriangleMesh with 3609 points and 7209 triangles.
run Poisson surface reconstruction
geometry::TriangleMesh with 3756 points and 7454 triangles.


temp = np.asarray(pcd.points)
new_points = duplicateSurface(temp)
new_pcl = o3d.geometry.PointCloud()
new_pcl.points = o3d.utility.Vector3dVector(np.concatenate((temp,new_points)))
new_pcl.normals = o3d.utility.Vector3dVector(np.concatenate((np.asarray(pcd.normals),-np.asarray(pcd.normals))))
new_pcl.paint_uniform_color([0.8, 0.8, 0.8])
#o3d.visualization.draw_geometries([new_pcl])
#o3d.io.write_point_cloud("./clean points/dupback_pose_1.ply", new_pcl)